This quickstart guide explains how to join two tables A and B using TF-IDF 
similarity measure. First, you need to import the required packages 
as follows (if you have installed **py_stringsimjoin** it will 
automatically install the dependencies **py_stringmatching** and **pandas**):

In [1]:
# Import libraries
import py_stringsimjoin as ssj
import py_stringmatching as sm
import pandas as pd
import os
import sys

In [2]:
print('python version: ' + sys.version)
print('py_stringsimjoin version: ' + ssj.__version__)
print('py_stringmatching version: ' + sm.__version__)
print('pandas version: ' + pd.__version__)

python version: 3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]
py_stringsimjoin version: 0.1.0
py_stringmatching version: 0.4.0
pandas version: 1.3.4


Joining two tables using TD-IDF measure typically consists of six steps:
1. Loading the input tables
2. Profiling the tables
3. Creating a tokenizer
4. Applying overlap filter
5. Creating the corpus for TF-IDF matcher 
6. Applying the TF-IDF matcher 

# 1. Loading the input tables

We begin by loading the two tables. For the purpose of this 
guide, we use the books dataset that comes with the package. 

In [3]:
# construct the path of the tables to be loaded. Since we are loading a 
# dataset from the package, we need to access the data from the path 
# where the package is installed. If you need to load your own data, you can directly
# provide your table path to the read_csv command.

table_A_path = "C:/Users/zoran/Desktop/W1.csv"
table_B_path = "C:/Users/zoran/Desktop/W2.csv"

In [4]:
# Load csv files as dataframes. Since we are reading a compressed csv file, 
# we provide the compression argument. If you are reading an uncompressed 
# csv file, you should not specify the compression argument.

A = pd.read_csv(table_A_path)
B = pd.read_csv(table_B_path)
print('Number of records in A: ' + str(len(A)))
print('Number of records in B: ' + str(len(B)))

Number of records in A: 814
Number of records in B: 143


In [5]:
A.head(1)

,ID,Name:,Hersteller:,Erzeugnis aus:,Region:,Wein:,Jahrgang:,Rebsorte:,Qualität:,Alkohol in %:,...,Herstellung:,Dekantieren:,Säure g/l:,Restzucker g/l:,Produkt aus biologischem Anbau:,Biokontrollstelle:,Diverses:,Zutaten:,Verkäufer:,Verkostungsnotiz:
0,1,Château Sansonnet 2019 Château Sansonnet Saint Emilion Grand Cru Classé,"Château Sansonnet, 1, lieu-dit Sansonnet, F-33330 Saint Emilion",Frankreich,"Bordeaux, Saint-Emilion",Rotwein,2019,"85% Merlot, 8% Cabernet Franc, 7% Cabernet Sauvignon",AOC,13.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"Lisa Perrotti-Brown, Wine Advocate, Juni 2020: „Opaque garnet-purple in color, the 2019 Sansonn..."


In [6]:
B.head(1)

,ID,Name:,Hersteller:,Erzeugnis aus:,Region:,Wein:,Jahrgang:,Rebsorte:,Qualität:,Alkohol in %:,...,Herstellung:,Dekantieren:,Säure g/l:,Restzucker g/l:,Produkt aus biologischem Anbau:,Biokontrollstelle:,Diverses:,Zutaten:,Verkäufer:,Verkostungsnotiz:
0,1,"2016 Cascina Castlèt Passum Barbera d'Asti D.O.C.G. Superiore 1,5l Magnum",Cascina Castlet,Italien,Piemont,Rotwein,2016.0,Barbera,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Dieser Barbera steht in einer Reihe mit Barolo und Barbaresco. Ein Triumph der traditionellen ro...


# 2. Profiling the tables

Before performing the join, we may want to profile the tables to 
know about the characteristics of the attributes. This can help identify:

a) unique attributes in the table which can be used as key attribute when performing 
   the join. A key attribute is needed to uniquely identify a tuple. 
   
b) the number of missing values present in each attribute. This can 
   help you in deciding the attribute on which to perform the join. 
   For example, an attribute with a lot of missing values may not be a good 
   join attribute. Further, based on the missing value information you 
   need to decide on how to handle missing values when performing the join 
   
You can profile the attributes in a table using the following command:

In [5]:
# profile attributes in table A
ssj.profile_table_for_join(A)

,Unique values,Missing values,Comments
Attribute,,,
ID,814 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
Name:,811 (99.63%),0 (0.0%),
Hersteller:,164 (20.15%),0 (0.0%),
Erzeugnis aus:,10 (1.23%),0 (0.0%),
Region:,64 (7.86%),0 (0.0%),
Wein:,9 (1.11%),2 (0.25%),Joining on this attribute will ignore 2 (0.25%) rows.
Jahrgang:,40 (4.91%),8 (0.98%),Joining on this attribute will ignore 8 (0.98%) rows.
Rebsorte:,141 (17.32%),36 (4.42%),Joining on this attribute will ignore 36 (4.42%) rows.
Qualität:,113 (13.88%),2 (0.25%),Joining on this attribute will ignore 2 (0.25%) rows.


In [7]:
# profile attributes in table B
ssj.profile_table_for_join(B)

,Unique values,Missing values,Comments
Attribute,,,
ID,143 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
Name:,137 (95.8%),0 (0.0%),
Hersteller:,12 (8.39%),67 (46.85%),Joining on this attribute will ignore 67 (46.85%) rows.
Erzeugnis aus:,5 (3.5%),3 (2.1%),Joining on this attribute will ignore 3 (2.1%) rows.
Region:,22 (15.38%),6 (4.2%),Joining on this attribute will ignore 6 (4.2%) rows.
Wein:,10 (6.99%),3 (2.1%),Joining on this attribute will ignore 3 (2.1%) rows.
Jahrgang:,13 (9.09%),76 (53.15%),Joining on this attribute will ignore 76 (53.15%) rows.
Rebsorte:,61 (42.66%),7 (4.9%),Joining on this attribute will ignore 7 (4.9%) rows.
Qualität:,1 (0.7%),143 (100.0%),Joining on this attribute will ignore 143 (100.0%) rows.


Based on the profile output, we find that the 'Title' attribute in both tables does 
not contain any missing values. Hence, for the purpose of this guide, we will now 
join tables A and B on 'Title' attribute using TF-IDF measure. Next, we need to decide 
on what threshold to use for the join. For this guide, we will use a threshold of 0.5. 
Specifically, the join will now find tuple pairs from A and B such that the TF-IDF score 
over the 'Title' attributes is at least 0.5.

Naively, performing the join will involve enumerating the cartesian product 
AxB (3022 x 3099 = 9365178) and computing TF-IDF score for every pair. But, this can be 
very time consuming. Hence, we can optimize by first appplying an overlap filter over tables 
A and B to find pairs sharing at least one token in the 'Title' attribute. The intuition here 
is that in order for TF-IDF score to be above zero, there must be at least one common token 
between the attributes. Finally, we apply the TF-IDF measure over the candidate pairs 
to obtain the join output.

# 3. Creating a tokenizer

Since TF-IDF measure treats input strings as bags of tokens, we 
need to select a tokenizer which can be used to tokenize each string 
into a bag of tokens. Currently, we support tokenizers from **py_stringmatching**
package which provides five different tokenizer types: alphabetical tokenizer, 
alphanumeric tokenizer, delimiter-based tokenizer, qgram tokenizer, 
and whitespace tokenizer.

For the purpose of this guide, we will use a whitespace tokenizer. Once
we have selected a tokenizer type, we need to create a tokenizer object as
shown below:

In [8]:
# create whitespace tokenizer for tokenizing 'Title' attribute
ws = sm.WhitespaceTokenizer()

ws.tokenize('The Maze Runner Series Complete Collection')

['The', 'Maze', 'Runner', 'Series', 'Complete', 'Collection']

# 4. Applying overlap filter

In [23]:
# create overlap filter with whitespace tokenizer and threshold of 1. 
of = ssj.OverlapFilter(ws, 5) 

# apply overlap filter to tables A and B to find tuple pairs 
# sharing at least 1 token in Title attribute
C = of.filter_tables(A, B, 'ID', 'ID', 'Name:', 'Name:', n_jobs=-1)

c:\projects\data integration\desktop\lib\site-packages\py_stringsimjoin\utils\validation.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if attr_type != pd.np.object:
c:\projects\data integration\desktop\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  projected_dataframe = dataframe[proj_attrs].dropna(0,


In [25]:
len(C)

86

In [26]:
C.head(5)

,_id,l_ID,r_ID
0,0,201,13
0,1,161,22
1,2,204,22
0,3,33,41
1,4,201,41


If you want to include pairs with missing value in the output, 
you need to set the **allow_missing** flag to True when creating
the overlap filter as shown below:

In [27]:
of = ssj.OverlapFilter(ws, 1, allow_missing=True) 

Now, when you apply the filter, pairs with missing values will also 
be included in the output.

# 5. Creating the corpus for TF-IDF matcher

The next step is to create the corpus required for TF-IDF measure. 
Specifically, the corpus consists of the list of tokens in the 'Title' 
attribute. The corpus can be created as follows:

In [28]:
# create a list of tokens
A_tokens = A['Name:'].apply(ws.tokenize).tolist()
B_tokens = B['Name:'].apply(ws.tokenize).tolist()

# merge both the lists of tokens to create the corpus
corpus = A_tokens + B_tokens

# 6. Applying the TF-IDF matcher 

Finally, you need to create and apply the TF-IDF matcher as shown below:

In [29]:
# create tf-idf object with the generated corpus
tfidf = sm.TfIdf(corpus, dampen=True)

In [36]:
# apply the matcher with a threshold of 0.5. This will find pairs from C 
# with TF-IDF score >= 0.5. Setting n_jobs=-1 exploits all CPU cores available.

output_pairs = ssj.apply_matcher(C, 'l_ID', 'r_ID', A, B, 'ID', 'ID', 'Name:', 'Name:',
                                 ws, tfidf.get_sim_score, 0.5, 
                                 l_out_attrs=['Name:'], r_out_attrs=['Name:'], n_jobs=-1)

In [37]:
len(output_pairs)

3

In [38]:
output_pairs.head()

,_id,l_ID,r_ID,l_Name:,r_Name:,_sim_score
0,74,713,101,Weingut K.-H. Milch 2020 Blanc de Noir Spätburgunder trocken,2019 Weingut Hain Spätburgunder Blanc de Noir Q.b.A. trocken,0.534284
0,75,692,103,Weingut Schloss Lieser 2019 Riesling Spätlese Piesporter Goldtröpfchen,2019 Weingut Hain Piesporter Goldtröpfchen Riesling Kabinett feinherb,0.510260
1,84,713,140,Weingut K.-H. Milch 2020 Blanc de Noir Spätburgunder trocken,2019 Weingut Hain Spätburgunder Blanc de Noir Q.b.A. trocken,0.534284


If you want to include pairs with missing value in the output, 
you need to set the **allow_missing** flag to True in the 
**apply_matcher** method.